In [1]:
import sqlalchemy as db
import numpy as np
import os
import config
import pandas as pd
pd.__version__

'0.24.2'

In [2]:
with open(os.path.expanduser("~/.pgpass"), 'r') as f:
    secrets = None
    for line in f:
        if config.wimbledon_config['host'] in line:
            secrets = line.strip().split(':')
            break
    
if secrets is None:
    raise ValueError('did not find '+config.wimbledon_config['host']+' in ~/.pgpass')

In [3]:
url = db.engine.url.URL(drivername=config.wimbledon_config['drivername'], 
                        username=secrets[-2], 
                        password=secrets[-1],
                        host=config.wimbledon_config['host'], 
                        database=config.wimbledon_config['database'])

engine = db.create_engine(url)

connection = engine.connect()
metadata = db.MetaData(schema='forecast')

In [4]:
def csv_to_sql(schema, table_name, usecols, parse_dates, ints_with_nan, index_col='id'):
    
    csv = pd.read_csv('../data/'+schema+'/'+table_name+'.csv',
                      usecols=usecols,
                      index_col=index_col,
                      parse_dates=parse_dates,
                      infer_datetime_format=True)
    
    for col in ints_with_nan:
        # Integer columns with NaN: Requires pandas 0.24 (otherwise ids end up as floats)
        csv[col] = csv[col].astype('Int64')
        
    if 'roles' in usecols:
        csv['roles'] = csv['roles'].str.replace(r"[\[\]\']", "", regex=True)
        csv.rename(columns={'roles':'role'},inplace=True)
        
        roles = pd.read_csv('../data/'+schema+'/roles.csv')
        csv['role'] = pd.merge(csv['role'], 
                               roles[['name','id']], 
                               how='left', left_on='role', right_on='name')['id']
        
        csv['role'] = csv['role'].astype('Int64')
        
    csv.columns = csv.columns.str.replace('.','_')
    
    metadata = db.MetaData(schema=schema)
   
    csv.to_sql(table_name, connection, schema=schema, if_exists='append')
    table = db.Table(table_name, metadata, autoload=True, autoload_with=engine)
    results = connection.execute(db.select([table])).fetchall()
    df = pd.DataFrame(results)
    df.columns = results[0].keys()
    
    print(repr(metadata.tables[schema+'.'+table_name]))
    display(df.head())

## Harvest

In [5]:
csv_to_sql('harvest', 'clients', 
           usecols=['id', 'name', 'is_active'],
           parse_dates=False,
           ints_with_nan=[],
           index_col='id')

Table('clients', MetaData(bind=None), Column('id', INTEGER(), table=<clients>, primary_key=True, nullable=False), Column('name', TEXT(), table=<clients>), Column('is_active', BOOLEAN(), table=<clients>), schema='harvest')


,id,name,is_active
0,7889179,Wellcome Trust,True
1,7738296,TIL,True
2,7738175,Leeds,True
3,7515887,Alan Turing Institute,True
4,7444895,SPF,True


In [6]:
csv_to_sql('harvest', 'projects', 
           usecols=['id', 'name', 'budget', 'code', 'starts_on', 'ends_on', 'client.id', 'notes', 'is_active'],
           parse_dates=['starts_on', 'ends_on'],
           ints_with_nan=['client.id'],
           index_col='id')

Table('projects', MetaData(bind=None), Column('id', INTEGER(), table=<projects>, primary_key=True, nullable=False), Column('name', TEXT(), table=<projects>), Column('budget', DOUBLE_PRECISION(precision=53), table=<projects>), Column('code', TEXT(), table=<projects>), Column('starts_on', DATE(), table=<projects>), Column('ends_on', DATE(), table=<projects>), Column('client_id', INTEGER(), ForeignKey('harvest.clients.id'), table=<projects>), Column('notes', TEXT(), table=<projects>), Column('is_active', BOOLEAN(), table=<projects>), schema='harvest')


,id,name,budget,code,starts_on,ends_on,client_id,notes,is_active
0,20843757,Urban Agriculture,NaN,None,None,None,7394384,None,True
1,20684616,Wimbledon Planner,NaN,None,None,None,7236211,Producing time-tracking and project forecastin...,True
2,20508323,Mini-bridge support,NaN,None,None,None,7394384,None,True
3,20463792,Uncertainty in Government Modelling,NaN,None,2019-04-01,2020-03-31,7394384,None,True
4,20431477,Urban systems resilience,NaN,None,2019-10-01,2019-12-31,7515887,None,True


In [7]:
csv_to_sql('harvest', 'roles', 
           usecols=['id', 'name'],
           parse_dates=False,
           ints_with_nan=[],
           index_col='id')

Table('roles', MetaData(bind=None), Column('id', INTEGER(), table=<roles>, primary_key=True, nullable=False), Column('name', TEXT(), table=<roles>), schema='harvest')


,id,name
0,510325,Community Manager
1,495088,example
2,493302,Research Project Manager
3,493301,Project Manager
4,463110,Research Software Engineer


In [8]:
csv_to_sql('harvest', 'users', 
           usecols=['id', 'first_name', 'last_name', 'email', 'roles', 'weekly_capacity', 'is_active', 'is_project_manager', 'is_contractor'],
           parse_dates=False,
           ints_with_nan=[],
           index_col='id')

Table('users', MetaData(bind=None), Column('id', INTEGER(), table=<users>, primary_key=True, nullable=False), Column('first_name', TEXT(), table=<users>), Column('last_name', TEXT(), table=<users>), Column('email', TEXT(), table=<users>), Column('role', INTEGER(), ForeignKey('harvest.roles.id'), table=<users>), Column('weekly_capacity', INTEGER(), table=<users>), Column('is_active', BOOLEAN(), table=<users>), Column('is_project_manager', BOOLEAN(), table=<users>), Column('is_contractor', BOOLEAN(), table=<users>), schema='harvest')


,id,first_name,last_name,email,role,weekly_capacity,is_active,is_project_manager,is_contractor
0,2764181,Gabriel,Hanganu,ghanganu@turing.ac.uk,None,0,True,False,False
1,2747278,Oliver,Forrest,oforrest@turing.ac.uk,None,144000,True,False,False
2,2603234,Jack,Roberts,jroberts@turing.ac.uk,None,144000,True,False,False
3,2593895,Amaani,Hoddoon,ahoddoon@turing.ac.uk,None,144000,True,False,False
4,2593869,Oscar,Giles,ogiles@turing.ac.uk,None,144000,True,False,False


In [9]:
csv_to_sql('harvest', 'tasks', 
           usecols=['id', 'name', 'is_active'],
           parse_dates=False,
           ints_with_nan=[],
           index_col='id')

Table('tasks', MetaData(bind=None), Column('id', INTEGER(), table=<tasks>, primary_key=True, nullable=False), Column('name', TEXT(), table=<tasks>), Column('is_active', BOOLEAN(), table=<tasks>), schema='harvest')


,id,name,is_active
0,12128239,HoloGraph,True
1,12050549,Wrattler racket backend,True
2,11872228,Reviewing,True
3,11872227,RepRes lunches,True
4,11872226,Content review,True


In [10]:
csv_to_sql('harvest', 'time_entries', 
           usecols=['id', 'user.id', 'project.id', 'task.id', 'spent_date', 'hours', 'notes'],
           parse_dates=['spent_date'],
           ints_with_nan=['user.id','project.id','task.id'],
           index_col='id')

Table('time_entries', MetaData(bind=None), Column('id', INTEGER(), table=<time_entries>, primary_key=True, nullable=False), Column('user_id', INTEGER(), ForeignKey('harvest.users.id'), table=<time_entries>), Column('project_id', INTEGER(), ForeignKey('harvest.projects.id'), table=<time_entries>), Column('task_id', INTEGER(), ForeignKey('harvest.tasks.id'), table=<time_entries>), Column('spent_date', DATE(), table=<time_entries>), Column('hours', DOUBLE_PRECISION(precision=53), table=<time_entries>), Column('notes', TEXT(), table=<time_entries>), schema='harvest')


,id,user_id,project_id,task_id,spent_date,hours,notes
0,994616111,2360118,18946187,10375716,2019-04-28,2.0,None
1,994616109,2360118,18946187,10375716,2019-04-27,2.0,None
2,994677858,2360132,18753173,10665526,2019-04-26,4.0,None
3,994677798,2360132,18793805,10375716,2019-04-26,2.0,None
4,994677797,2360132,18946187,10375716,2019-04-26,2.0,None


In [11]:
csv_to_sql('harvest', 'user_assignments', 
           usecols=['id', 'user.id', 'project.id', 'is_active', 'is_project_manager'],
           parse_dates=False,
           ints_with_nan=['user.id','project.id'],
           index_col='id')

Table('user_assignments', MetaData(bind=None), Column('id', INTEGER(), table=<user_assignments>, primary_key=True, nullable=False), Column('user_id', INTEGER(), ForeignKey('harvest.users.id'), table=<user_assignments>), Column('project_id', INTEGER(), ForeignKey('harvest.projects.id'), table=<user_assignments>), Column('is_active', BOOLEAN(), table=<user_assignments>), Column('is_project_manager', BOOLEAN(), table=<user_assignments>), schema='harvest')


,id,user_id,project_id,is_active,is_project_manager
0,197387056,2443850,18946199,True,False
1,196822066,2603234,18753173,True,False
2,196269279,2463055,19471496,True,False
3,196227290,2392818,20843757,True,True
4,196213146,2593869,19930508,True,False


In [12]:
csv_to_sql('harvest', 'task_assignments', 
           usecols=['id', 'task.id', 'project.id'],
           parse_dates=False,
           ints_with_nan=['task.id','project.id'],
           index_col='id')

Table('task_assignments', MetaData(bind=None), Column('id', INTEGER(), table=<task_assignments>, primary_key=True, nullable=False), Column('project_id', INTEGER(), ForeignKey('harvest.projects.id'), table=<task_assignments>), Column('task_id', INTEGER(), ForeignKey('harvest.tasks.id'), table=<task_assignments>), schema='harvest')


,id,project_id,task_id
0,225689177,18650735,12128239
1,224600345,20843757,10638208
2,224600344,20843757,10911795
3,224600343,20843757,11872225
4,224600342,20843757,10375718


## Forecast

In [13]:
csv_to_sql('forecast', 'clients', 
           usecols=['id', 'name', 'harvest_id', 'archived'],
           parse_dates=False,
           ints_with_nan=['harvest_id'],
           index_col='id')

Table('clients', MetaData(bind=None), Column('id', INTEGER(), table=<clients>, primary_key=True, nullable=False), Column('name', TEXT(), table=<clients>), Column('harvest_id', INTEGER(), ForeignKey('harvest.clients.id'), table=<clients>), Column('archived', BOOLEAN(), table=<clients>), schema='forecast')


,id,name,harvest_id,archived
0,732424,The Alan Turing Institute,NaN,False
1,732425,Intel,NaN,False
2,744985,Wrattler,NaN,False
3,745536,Wilfrid Kendall (Warwick University),NaN,False
4,745537,HSBC,NaN,False


In [14]:
csv_to_sql('forecast', 'projects', 
           usecols=['id', 'name', 'code', 'start_date', 'end_date', 'client_id', 'harvest_id', 'notes', 'archived'],
           parse_dates=['start_date', 'end_date'],
           ints_with_nan=['client_id','harvest_id'],
           index_col='id')

Table('projects', MetaData(bind=None), Column('id', INTEGER(), table=<projects>, primary_key=True, nullable=False), Column('name', TEXT(), table=<projects>), Column('code', TEXT(), table=<projects>), Column('start_date', DATE(), table=<projects>), Column('end_date', DATE(), table=<projects>), Column('client_id', INTEGER(), ForeignKey('forecast.clients.id'), table=<projects>), Column('harvest_id', INTEGER(), ForeignKey('harvest.projects.id'), table=<projects>), Column('notes', TEXT(), table=<projects>), Column('archived', BOOLEAN(), table=<projects>), schema='forecast')


,id,name,code,start_date,end_date,client_id,harvest_id,notes,archived
0,1684536,Time Off,None,None,None,NaN,NaN,None,False
1,1684539,Sargasso (Intel Sparse),R-INT-001,2018-04-01,2018-12-31,769477.0,18266019.0,None,False
2,1723251,AIDA,None,2018-04-01,2019-03-31,761645.0,18644291.0,None,False
3,1723252,DetectorChecker,None,2018-08-20,2019-01-31,761647.0,18644652.0,None,False
4,1723253,HSBC EDS,R-HSB-001,2018-04-01,2018-10-31,769469.0,18644656.0,None,False


In [15]:
csv_to_sql('forecast', 'roles', 
           usecols=['id', 'name', 'harvest_role_id'],
           parse_dates=False,
           ints_with_nan=['harvest_role_id'],
           index_col='id')

Table('roles', MetaData(bind=None), Column('id', INTEGER(), table=<roles>, primary_key=True, nullable=False), Column('name', TEXT(), table=<roles>), Column('harvest_role_id', INTEGER(), ForeignKey('harvest.roles.id'), table=<roles>), schema='forecast')


,id,name,harvest_role_id
0,170095,Research Data Scientist,450792
1,170096,Senior Research Software Engineer,450793
2,170097,Senior Research Data Scientist,450794
3,170099,Principal Research Data Scientist,450796
4,170100,Director of Research Engineering,450797


In [16]:
csv_to_sql('forecast', 'people', 
           usecols=['id', 'first_name', 'last_name', 'email', 'roles', 'harvest_user_id', 'login', 'subscribed', 'admin',
                   'archived', 'weekly_capacity', 'working_days.monday', 'working_days.tuesday', 'working_days.wednesday',
                   'working_days.thursday', 'working_days.friday', 'working_days.saturday', 'working_days.sunday'],
           parse_dates=False,
           ints_with_nan=['harvest_user_id'],
           index_col='id')

Table('people', MetaData(bind=None), Column('id', INTEGER(), table=<people>, primary_key=True, nullable=False), Column('first_name', TEXT(), table=<people>), Column('last_name', TEXT(), table=<people>), Column('email', TEXT(), table=<people>), Column('role', INTEGER(), table=<people>), Column('harvest_user_id', INTEGER(), ForeignKey('harvest.users.id'), table=<people>), Column('login', TEXT(), table=<people>), Column('subscribed', BOOLEAN(), table=<people>), Column('admin', BOOLEAN(), table=<people>), Column('archived', BOOLEAN(), table=<people>), Column('weekly_capacity', INTEGER(), table=<people>), Column('working_days_monday', BOOLEAN(), table=<people>), Column('working_days_tuesday', BOOLEAN(), table=<people>), Column('working_days_wednesday', BOOLEAN(), table=<people>), Column('working_days_thursday', BOOLEAN(), table=<people>), Column('working_days_friday', BOOLEAN(), table=<people>), Column('working_days_saturday', BOOLEAN(), table=<people>), Column('working_days_sunday', BOOLEA

,id,first_name,last_name,email,role,harvest_user_id,login,subscribed,admin,archived,weekly_capacity,working_days_monday,working_days_tuesday,working_days_wednesday,working_days_thursday,working_days_friday,working_days_saturday,working_days_sunday
0,399979,Oliver,Strickson,ostrickson@turing.ac.uk,None,2315537.0,enabled,True,True,False,NaN,True,True,True,True,True,False,False
1,408178,Angus,Williams,awilliams@turing.ac.uk,None,NaN,disabled,False,False,False,NaN,True,True,True,True,True,False,False
2,408179,David,Beavan,dbeavan@turing.ac.uk,None,2360118.0,enabled,False,True,False,NaN,True,True,True,True,True,False,False
3,408180,Evelina,Gabasova,egabasova@turing.ac.uk,None,2360119.0,enabled,False,True,False,NaN,True,True,True,True,True,False,False
4,408181,Giovanni,Colavizza,gcolavizza@turing.ac.uk,None,2360121.0,enabled,False,True,False,NaN,True,True,True,True,True,False,False


In [17]:
csv_to_sql('forecast', 'placeholders', 
           usecols=['id', 'name', 'roles', 'archived'],
           parse_dates=False,
           ints_with_nan=[],
           index_col='id')

Table('placeholders', MetaData(bind=None), Column('id', INTEGER(), table=<placeholders>, primary_key=True, nullable=False), Column('name', TEXT(), table=<placeholders>), Column('role', INTEGER(), table=<placeholders>), Column('archived', BOOLEAN(), table=<placeholders>), schema='forecast')


,id,name,role,archived
0,23092,Resource Required 1,None,False
1,23093,Newcastle 01,None,False
2,23094,Edinburgh 01,None,False
3,24360,Birmingham 01,None,False
4,24361,SPF RA 01,None,False


In [18]:
csv_to_sql('forecast', 'milestones', 
           usecols=['id', 'date', 'project_id'],
           parse_dates=['date'],
           ints_with_nan=['project_id'],
           index_col='id')

Table('milestones', MetaData(bind=None), Column('id', INTEGER(), table=<milestones>, primary_key=True, nullable=False), Column('date', DATE(), table=<milestones>), Column('project_id', INTEGER(), ForeignKey('forecast.projects.id'), table=<milestones>), schema='forecast')


,id,date,project_id
0,1727379,2018-08-30,1684539
1,1906966,2019-01-02,1823898
2,2088931,2019-03-19,1969208


In [19]:
csv_to_sql('forecast', 'assignments', 
           usecols=['id', 'person_id', 'placeholder_id', 'project_id', 'start_date','end_date','allocation','notes'],
           parse_dates=['start_date','end_date'],
           ints_with_nan=['person_id','placeholder_id','project_id'],
           index_col='id')

Table('assignments', MetaData(bind=None), Column('id', INTEGER(), table=<assignments>, primary_key=True, nullable=False), Column('person_id', INTEGER(), ForeignKey('forecast.people.id'), table=<assignments>), Column('placeholder_id', INTEGER(), ForeignKey('forecast.placeholders.id'), table=<assignments>), Column('project_id', INTEGER(), ForeignKey('forecast.projects.id'), table=<assignments>), Column('start_date', DATE(), table=<assignments>), Column('end_date', DATE(), table=<assignments>), Column('allocation', INTEGER(), table=<assignments>), Column('notes', TEXT(), table=<assignments>), schema='forecast')


,id,person_id,placeholder_id,project_id,start_date,end_date,allocation,notes
0,19314451,408190.0,NaN,1723252,2018-11-01,2018-11-30,14400,None
1,19314475,408190.0,NaN,1723263,2018-11-01,2018-12-31,14400,None
2,19314509,408187.0,NaN,1723263,2018-11-01,2018-12-31,14400,None
3,19314518,408187.0,NaN,1723251,2018-11-01,2019-03-31,14400,None
4,19314525,408186.0,NaN,1723251,2018-11-01,2019-03-31,14400,None
